In [ ]:
import pandas as pd
from tqdm import tqdm
import pickle

# Datos sin correccion monetaria

In [ ]:
years = [2014, 2015, 2016, 2017, 2018, 2019]

df = pd.DataFrame()

pbar = tqdm(total=len(years))
for i, year in enumerate(years):
    # carga de datos, skiprows=1 porque hay una fila con el año
    _df = pd.read_excel(f"../Datos/02_intermediate/sinim/sinim_{year}.xlsx", engine="openpyxl", skiprows=1)

    # cambio de nombre a las primeras dos columnas que corresponden al codigo y municipio
    _df.rename(mapper={"Unnamed: 0":"codigo", "Unnamed: 1":"municipio"}, axis=1, inplace=True)
    
    # se elimina la primera fila pues tiene solo el año
    _df.drop(0, inplace=True)

    # crear un glosario de datos
    if i==0:
        col_name_description = {col.split(" ")[0]: col.replace(col.split(" ")[0], "") for col in _df.columns if not (col=="codigo" or col=="municipio")}
        with open("../Datos/09_dictionaries/glosario_columnas_sinim.pkl", "wb") as file:
            pickle.dump(col_name_description, file)
    
    # cambiar nombres de las columnas
    col_name_mapper = {col: col.split(" ")[0] for col in _df.columns if not (col=="codigo" or col=="municipio")}
    _df.rename(mapper=col_name_mapper, axis=1, inplace=True)

    # transformar a datos numéricos
    for col in df.columns:
        if not(col=="codigo" or col=="municipio" or col=="año"):
            if "(S-N)" not in col_name_description[col]:
                _df[col] = pd.to_numeric(_df[col], errors="coerce")

    # agregar año
    _df["año"] = year
    
    # concatenar
    df = pd.concat([df, _df], axis=0)
    pbar.update(1)

pbar.close()

In [ ]:
df.reset_index(drop=True)
df.to_csv(f"../Datos/03_primary/sinim_{min(years)}_{max(years)}_consolidado.csv")

# Datos con correccion monetaria

In [ ]:
years = [2014, 2015, 2016, 2017, 2018, 2019]

df = pd.DataFrame()

pbar = tqdm(total=len(years))
for i, year in enumerate(years):
    # carga de datos, skiprows=1 porque hay una fila con el año
    _df = pd.read_excel(f"../Datos/02_intermediate/sinim/sinim_{year}_correccion.xlsx", engine="openpyxl", skiprows=1)

    # cambio de nombre a las primeras dos columnas que corresponden al codigo y municipio
    _df.rename(mapper={"Unnamed: 0":"codigo", "Unnamed: 1":"municipio"}, axis=1, inplace=True)
    
    # se elimina la primera fila pues tiene solo el año
    _df.drop(0, inplace=True)

    # crear un glosario de datos
    if i==0:
        col_name_description = {col.split(" ")[0]: col.replace(col.split(" ")[0], "") for col in _df.columns if not (col=="codigo" or col=="municipio")}
        with open("../Datos/09_dictionaries/glosario_columnas_sinim.pkl", "wb") as file:
            pickle.dump(col_name_description, file)
    
    # cambiar nombres de las columnas
    col_name_mapper = {col: col.split(" ")[0] for col in _df.columns if not (col=="codigo" or col=="municipio")}
    _df.rename(mapper=col_name_mapper, axis=1, inplace=True)

    # transformar a datos numéricos
    for col in df.columns:
        if not(col=="codigo" or col=="municipio" or col=="año"):
            if "(S-N)" not in col_name_description[col]:
                _df[col] = pd.to_numeric(_df[col], errors="coerce")

    # agregar año
    _df["año"] = year
    
    # concatenar
    df = pd.concat([df, _df], axis=0)
    pbar.update(1)

pbar.close()

In [ ]:
df.reset_index(drop=True)
df.to_csv(f"../Datos/03_primary/sinim_con_correccion_{min(years)}_{max(years)}_consolidado.csv")